## Some useful analogies to SQL operations

Data set is a list of two columns, viz., userid and app name. See venn_sample_gen.py to generate this set.

Sample given below.

| userid  | app        | 
|---------|------------| 
| u000001 | ola        | 
| u000002 | freecharge | 
| u000002 | mobikwik   | 
| u000002 | fastcab    | 
| u000003 | uber       | 
| u000003 | ola        | 
| u000003 | freecharge | 
| u000004 | ola        | 
| u000004 | mobikwik   | 
| u000004 | uber       | 
| u000004 | fastcab    | 
| u000004 | freecharge | 


In [1]:
import numpy as np
import pandas as pd
from pandas import Series, DataFrame, Panel
from odo import odo

In [7]:
# df = odo('venn_sample_gen.csv', pd.DataFrame) 
df = pd.DataFrame(
[
('u000001','ola'),
('u000002','freecharge'),
('u000002','mobikwik'),
('u000002','fastcab'),
('u000003','uber'),
('u000003','ola'),
('u000003','freecharge'),
('u000004','ola'),
('u000004','mobikwik'),
('u000004','uber'),
('u000004','fastcab'),
('u000004','freecharge')
], columns=['userid','app']
)
df

,userid,app
0,u000001,ola
1,u000002,freecharge
2,u000002,mobikwik
3,u000002,fastcab
4,u000003,uber
5,u000003,ola
6,u000003,freecharge
7,u000004,ola
8,u000004,mobikwik
9,u000004,uber


#### how many users?
```sql
select count(distinct userid) from userapps
```

In [8]:
len(df.groupby('userid').groups)

4

#### users with more than X apps?
```sql
select userid, count(1) from userapps group by userid having count(1) > X order by 2 desc;
```
Let us assume X=2

In [11]:
MIN_APPS = 2
# all these below can be in one chained line; breaking it up for better readability
gf = DataFrame({'count':df.groupby('userid')['app'].count()})
gf = gf.reset_index().set_index('userid', drop=True)
gf = gf.query("count > %d" % MIN_APPS).sort_values('count', ascending=False)
gf

,count
userid,
u000004,5
u000002,3
u000003,3


### how many users have both uber and ola apps?
```sql
select o.userid from userapps o where o.app='uber' and exists 
   (select 1 from userapps i where i.userid=o.userid and i.app='ola')
```
I know this is better done using a join than exists; but for illustration, this is easier.

In [12]:
ola_and_uber = pd.merge(df.query("app == 'uber'"), df.query("app == 'ola'"), on='userid')
ola_and_uber

,userid,app_x,app_y
0,u000003,uber,ola
1,u000004,uber,ola


In [13]:
# now add a field that shows the other apps these particular set has
pd.merge(ola_and_uber, df.query("app not in ('ola','uber')"), on='userid')

,userid,app_x,app_y,app
0,u000003,uber,ola,freecharge
1,u000004,uber,ola,mobikwik
2,u000004,uber,ola,fastcab
3,u000004,uber,ola,freecharge


### how many users use ola, but not uber?
```sql
select o.userid from userapps o where o.app='ola' and not exists 
   (select 1 from userapps i where i.userid=o.userid and i.app='uber')
```

In [14]:
bothusers = df.loc[df['app'].isin(['uber','ola'])]
uberusers  = df.loc[df['app'] == 'uber']
bothusers.set_index("userid").index.difference(uberusers.set_index("userid").index)

Index([u'u000001'], dtype='object', name=u'userid')

### how do we transform data into a pivot for NoSQL fans?

In [15]:
userapps = DataFrame({'apps':df.groupby('userid')['app'].apply(tuple)})
userapps

,apps
userid,
u000001,"(ola,)"
u000002,"(freecharge, mobikwik, fastcab)"
u000003,"(uber, ola, freecharge)"
u000004,"(ola, mobikwik, uber, fastcab, freecharge)"
